## 1. 유사도

pororo를 안쓰신다기에, sklearn 벡터라이저를 활용하여 전처리를 했습니다.
자카드, 유클리디안, 멘허튼으로 구성해봤습니다.

In [ ]:
sentence = ( "휴일 인 오늘 도 서쪽 을 중심 으로 폭염 이 이어졌는데요, 내일 은 반가운 비 소식 이 있습니다.",
	     "폭염 을 피해서 휴일 에 놀러왔다가 갑작스런 비 로 인해 망연자실 하고 있습니다.")


from sklearn.feature_extraction.text import TfidfVectorizer

# 객체 생성
tfidf_vectorizer = TfidfVectorizer()

# 문장 벡터화 진행 - 이것을 쓸겁니다.
tfidf_matrix = tfidf_vectorizer.fit_transform(sentence)

# 각 단어
text = tfidf_vectorizer.get_feature_names()

# 각 단어의 벡터 값
idf = tfidf_vectorizer.idf_
idf

print("첫번째 문장",tfidf_matrix[0:1])
print("두번째 문장",tfidf_matrix[1:2])

In [ ]:
# 1.유클리디안 유사도
from sklearn.metrics.pairwise import euclidean_distances
import numpy as np

def l1_normalize(v): # 일반화
    norm = np.sum(v)
    return v / norm
    
# 벡터라이징한 2개의 문장 전부 노말라이즈.
tfidf_norm_l1 = l1_normalize(tfidf_matrix) 

euclidean_distances(tfidf_norm_l1[0:1], tfidf_norm_l1[1:2])

In [ ]:
# 2. 맨하탄 유사도
from sklearn.metrics.pairwise import manhattan_distances

import numpy as np

def l1_normalize(v):
    norm = np.sum(v)
    return v / norm 
  
# L1 정규화  
tfidf_norm_l1 = l1_normalize(tfidf_matrix)

manhattan_distances(tfidf_norm_l1[0:1], tfidf_norm_l1[1:2])

In [ ]:
# 3. 자카드 유사도

# 위에 sentence 문장 2개를 활용했습니다.

data1 = sentence[0].split()
data2 = sentence[1].split()
data1
intd = set(data1) & set(data2) # 교집합
intd
un = set(data1) | set(data2) # 합집합
un
len(intd)/len(un) # data1과 dat2의 자카드 유사도 : 0.7

## 2. Text classification
데이터셋이 너무 크면, 기간내에 전달 못할 것 같아서 네이버 댓글 감성분석으로 구성해봤습니다.


- 데이터 설명

네이버 영화 리뷰 데이터
총 200,000개 리뷰로 구성된 데이터로 영화 리뷰에 대한 텍스트와 해당 리뷰가 긍정인 경우 1, 부정인 경우 0을 표시한 레이블

In [ ]:
!pip install konlpy

In [ ]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
from konlpy.tag import Okt
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

In [ ]:
train_data = pd.read_table('ratings_train.txt')
test_data = pd.read_table('ratings_test.txt')

In [ ]:
train_data[:5] # 상위 5개 출력

In [ ]:
# document 열과 label 열의 중복을 제외한 값의 개수
train_data['document'].nunique(), train_data['label'].nunique()

In [ ]:
# document 열의 중복 제거
train_data.drop_duplicates(subset=['document'], inplace=True)

In [ ]:
#train_data에서 해당 리뷰의 긍, 부정 유무 갯수
train_data['label'].value_counts().plot(kind = 'bar')


In [ ]:
# 데이터에 널값이 존재합니다
print(train_data.isnull().sum())

In [ ]:
train_data = train_data.dropna(how = 'any') # Null 값이 존재하는 행 제거
print(train_data.isnull().values.any()) # Null 값이 존재하는지 확인

In [ ]:
# 한글과 공백을 제외하고 모두 제거
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
train_data[:5]

In [ ]:
# 네이버 영화 리뷰는 한글이 아니더라도 영어, 숫자, 특수문자로도 리뷰를 업로드할 수 있습니다. 
# 다시 말해 기존에 한글이 없는 리뷰였다면 더 이상 아무런 값도 없는 빈(empty) 값이 되었을 것입니다. 
# train_data에 공백(whitespace)만 있거나 빈 값을 가진 행이 있다면 Null 값으로 변경하도록 하고, Null 값이 존재하는지 확인해보겠습니다.

train_data['document'] = train_data['document'].str.replace('^ +', "") # white space 데이터를 empty value로 변경
train_data['document'].replace('', np.nan, inplace=True)
print(train_data.isnull().sum())

In [ ]:
# 제거해줍니다.
train_data = train_data.dropna(how = 'any')
print(len(train_data))

In [ ]:
# 테스트 데이터에 앞서 진행한 전처리 과정을 동일하게 진행합니다.

test_data.drop_duplicates(subset = ['document'], inplace=True) # document 열에서 중복인 내용이 있다면 중복 제거
test_data['document'] = test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
test_data['document'] = test_data['document'].str.replace('^ +', "") # 공백은 empty 값으로 변경
test_data['document'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
test_data = test_data.dropna(how='any') # Null 값 제거
print('전처리 후 테스트용 샘플의 개수 :',len(test_data))

In [ ]:
# 토큰화를 위한 형태소 분석기는 KoNLPy의 Okt를 사용합니다.

X_train = []

stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
okt = Okt()
for sentence in tqdm(train_data['document']):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    X_train.append(stopwords_removed_sentence)

In [ ]:
#  테스트 데이터에 대해서도 동일하게 토큰화를 해줍니다.

X_test = []
for sentence in tqdm(test_data['document']):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    X_test.append(stopwords_removed_sentence)

In [ ]:
#기계가 텍스트를 숫자로 처리할 수 있도록 훈련 데이터와 테스트 데이터에 정수 인코딩을 수행
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

In [ ]:
# 단어 집합이 생성되는 동시에 각 단어에 고유한 정수가 부여
print(tokenizer.word_index)

In [ ]:
# 등장 빈도수가 3회 미만인 단어들이 이 데이터에서 얼만큼의 비중을 차지하는지 확인
threshold = 3
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)


In [ ]:
# 등장 빈도가 threshold 값인 3회 미만. 
# 즉, 2회 이하인 단어들은 단어 집합에서 무려 절반 이상을 차지합니다. 
# 하지만, 실제로 훈련 데이터에서 등장 빈도로 차지하는 비중은 상대적으로 매우 적은 수치인 1.87%밖에 되지 않습니다. 
# 아무래도 등장 빈도가 2회 이하인 단어들은 자연어 처리에서 별로 중요하지 않을 듯 합니다. 
# 그래서 이 단어들은 정수 인코딩 과정에서 배제시키겠습니다.

# 등장 빈도수가 2이하인 단어들의 수를 제외한 단어의 개수를 단어 집합의 최대 크기로 제한하겠습니다.

# 전체 단어 개수 중 빈도수 2이하인 단어는 제거.
# 0번 패딩 토큰을 고려하여 + 1
vocab_size = total_cnt - rare_cnt + 1
print('단어 집합의 크기 :',vocab_size)

In [ ]:
# 단어 집합의 크기는 19,416개입니다. 이를 케라스 토크나이저의 인자로 넘겨주고 텍스트 시퀀스를 정수 시퀀스로 변환합니다.

tokenizer = Tokenizer(vocab_size) 
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [ ]:
# train_data에서 y_train과 y_test를 별도로 저장해줍니다.

y_train = np.array(train_data['label'])
y_test = np.array(test_data['label'])

In [ ]:
# 빈 샘플(empty samples) 제거

# 전체 데이터에서 빈도수가 낮은 단어가 삭제되었다는 것은 빈도수가 낮은 단어만으로 
# 구성되었던 샘플들은 빈(empty) 샘플이 되었다는 것을 의미합니다. 
# 빈 샘플들은 어떤 레이블이 붙어있던 의미가 없으므로 빈 샘플들을 제거해주는 작업을 하겠습니다.
# 각 샘플들의 길이를 확인해서 길이가 0인 샘플들의 인덱스를 받아오겠습니다.

drop_train = [index for index, sentence in enumerate(X_train) if len(sentence) < 1]

# 빈 샘플들을 제거
X_train = np.delete(X_train, drop_train, axis=0)
y_train = np.delete(y_train, drop_train, axis=0)

# 빈 샘플들을 제거한 후의 샘플 개수
print(len(X_train))
print(len(y_train))

In [ ]:
# 서로 다른 길이의 샘플들의 길이를 동일하게 맞춰주는 패딩 작업을 진행해보겠습니다.
# 전체 데이터에서 가장 길이가 긴 리뷰와 전체 데이터의 길이 분포를 알아보겠습니다.

print('리뷰의 최대 길이 :',max(len(review) for review in X_train))
print('리뷰의 평균 길이 :',sum(map(len, X_train))/len(X_train))
plt.hist([len(review) for review in X_train], bins=50)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show()

In [ ]:
# 가장 긴 리뷰의 길이는 69이며, 
# 그래프를 봤을 때 전체 데이터의 길이 분포는 대체적으로 약 11내외의 길이를 가지는 것을 볼 수 있습니다. 
# 모델이 처리할 수 있도록 X_train과 X_test의 모든 샘플의 길이를 특정 길이로 동일하게 맞춰줄 필요가 있습니다. 
# 특정 길이 변수를 max_len으로 정합니다. 
# 대부분의 리뷰가 내용이 잘리지 않도록 할 수 있는 최적의 max_len의 값은 몇일까요?
# 전체 샘플 중 길이가 max_len 이하인 샘플의 비율이 몇 %인지 확인하는 함수를 만듭니다.

def below_threshold_len(max_len, nested_list):
  count = 0
  for sentence in nested_list:
    if(len(sentence) <= max_len):
        count = count + 1
  print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (count / len(nested_list))*100))

In [ ]:
# 위의 분포 그래프를 봤을 때, max_len = 30이 적당할 것 같습니다.
max_len = 30
below_threshold_len(max_len, X_train)


In [ ]:
# 전체 훈련 데이터 중 약 94%의 리뷰가 30이하의 길이를 가지는 것을 확인했습니다.
# 모든 샘플의 길이를 30으로 맞추겠습니다.

X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

In [ ]:
import re
from tensorflow.keras.layers import Embedding, Dense, LSTM, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

embedding_dim = 100
hidden_units = 128

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(Bidirectional(LSTM(hidden_units))) # Bidirectional LSTM을 사용
model.add(Dense(1, activation='sigmoid'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=256, validation_split=0.2)

In [ ]:
# 조기 종료 조건에 따라서 8 에포크에서 훈련이 멈췄습니다. 
# 훈련이 다 되었다면 테스트 데이터에 대해서 정확도를 측정할 차례입니다. 
# 훈련 과정에서 검증 데이터의 정확도가 가장 높았을 때 저장된 모델인 'best_model.h5'를 로드합니다.

loaded_model = load_model('best_model.h5')
print("\n 테스트 정확도(accuracy): %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

In [ ]:
# Predict
y_prediction = loaded_model.predict(X_test)

# 이진화
pred_class = np.where(y_prediction > 0.5, 1 , 0)

from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt

cm = confusion_matrix(y_true=y_test, y_pred=pred_class)

In [ ]:
# confusion matrix 결과
cm

In [ ]:
# Test 코드입니다.

def sentiment_predict(new_sentence):
    new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', new_sentence)
    new_sentence = okt.morphs(new_sentence, stem=True) # 토큰화
    new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거
    encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
    pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩
    score = float(loaded_model.predict(pad_new)) # 예측
    if(score > 0.5):
        print("{:.2f}% 확률로 긍정 리뷰입니다.\n".format(score * 100))
    else:
        print("{:.2f}% 확률로 부정 리뷰입니다.\n".format((1 - score) * 100))


sentiment_predict('너무 재미없었다. 다시는 안보겠다.')

## 3. Text Summarization 

Transformer 기반 모델인 Text-to-Text Transfer Transformer (T5)를 사용할 예정입니다. T5는 encoder-decoder 구조를 갖는 모델로 요약, 번역등 다양한 task에서 활용됩니다.

In [ ]:
# 필요한 라이브러리들 설치
!pip install transformers==4.20.0
!pip install nltk
!pip install -U nltk
!pip install rouge-score
!pip install keras_nlp==0.3.0
!pip install datasets
!pip install huggingface-hub

In [ ]:
import os
import logging
from pprint import pprint
import json
import nltk
import numpy as np
import tensorflow as tf
from tensorflow import keras

# 에러 메세지만 로깅
tf.get_logger().setLevel(logging.ERROR)

os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
# 필요한 옵션들 셋팅

# train과 test 데이터셋으로 나누는 비율
TRAIN_TEST_SPLIT = 0.1

MAX_INPUT_LENGTH = 512  # encoder에 들어갈 max input 길이
MIN_TARGET_LENGTH = 5  # decoder에 들어갈 min input 길이
MAX_TARGET_LENGTH = 128  # decoder에 들어갈 max input 길이
BATCH_SIZE = 8  # 모델 학습에 사용할 batch siz 크기
LEARNING_RATE = 2e-5  # 모델 학습에 사용할 learning rate
MAX_EPOCHS = 5  # 모델 학습에 사용할 epoch수

# Hugging Face Model Hub로 부터 가져올 모델명
MODEL_CHECKPOINT = "psyche/KoT5-summarization"
     

Load the dataset

전체 데이터셋은 AI Hub에서 다운받으실 수 있습니다. 

https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=97

우선 구글 드라이브에서 데이터 샘플을 가져옵니다.

다운 받은 데이터셋을 Hugging Face Datasets 포멧으로 변환합니다.


abstractive가 요약문이고 article_original이 원본 본문입니다.

In [ ]:
!gdown 1S5kUCc-u-F2w5JOgS81w2ht6Wmcjo7-y
!ls
!apt-get install jq
!head -n 1 ./sample.jsonl | jq '.'

데이터의 키 값을 article_original은 document로, abstractive는 summary로 변경합니다.

In [ ]:
# google drive에 있는 데이터를 사용하기 위해 접근

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
"""
from datasets import Dataset, load_dataset
ds_path = "./sample.jsonl"

# 데이터 담을 배열 선언
_id, document, summary = [], [], []

# 제이슨 읽어오기
with open(ds_path, 'r') as f:
  while True:
    line = f.readlines()
    if not line: break
    doc = json.loads(line)
    _id.append(doc['id'])
    document.append(" ".join(doc['article_original']))
    summary.append(doc['abstractive'])

# raw_datasets = load_dataset("xsum", split="train")
raw_datasets = Dataset.from_dict({"id":_id,
                                  "document":document,
                                  "summary":summary})
print(raw_datasets[0])
"""
from datasets import Dataset
import json
ds_path = "/content/drive/MyDrive/숨고/이인균님(인물 분류, 문서 요약)/법률_train_original/train_original.json"

# 데이터 담을 배열 선언
_id, document, summary = [], [], []

with open(ds_path, "r") as st_json:
    doc = json.load(st_json)

for i in doc['documents']:
    _id.append(i['id'])
    document.append(' '.join([sent['sentence'] for sent in i['text'][0]]))
    summary.append(i['abstractive'][0])

# raw_datasets = load_dataset("xsum", split="train")
raw_datasets = Dataset.from_dict({"id":_id,
                                  "document":document,
                                  "summary":summary})

print(raw_datasets[0])

In [ ]:
# 데이터를 학습용과 테스트용으로 스플릿합니다.
raw_datasets = raw_datasets.train_test_split(
    train_size=1-TRAIN_TEST_SPLIT, test_size=TRAIN_TEST_SPLIT
)
raw_datasets

In [ ]:
# Hugging Face Transformers의 Tokenizer 라이브러리로 MODEL_CHECKPOINT에 저장되어 있는 토크나이저를 로딩합니다. 
# Hugging Face Model Hub로 부터 가져올 토크나이징 모델명이 위에서 선언한 MODEL_CHECKPOINT, 즉 psyche/KoT5-summarization을 말합니다.
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

In [ ]:
# T5에서는 task를 prefix로 명시해주는 경우가 있습니다. 요약이라는 prefix를 추가해줍니다.
if MODEL_CHECKPOINT in ["psyche/KoT5-summarization", "t5-small", "t5-base", "t5-large", "t5-3b", "t5-11b"]:
    prefix = "요약: "
else:
    prefix = ""

prefix

In [ ]:
# 데이터 전처리를 위한 함수를 선언합니다.
# 전처리 함수에서는 prefix 추가 및 tokenization을 진행합니다.
# 이 과정에서 token_tpye_ids, attention_mask도 자동으로 생성됩니다.

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=MAX_INPUT_LENGTH, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["summary"], max_length=MAX_TARGET_LENGTH, truncation=True
        )

    model_inputs["labels"] = labels["input_ids"]

    return model_inputs


In [ ]:
# 이전에 생성했던 dataset 오브젝트에 map 메소드를 통해 preprocess_function 함수를 적용합니다.
# map을 사용할 경우 dataset내에 쪼개져서 구성되어 있는 train, validation, test 데이터셋에도 한번에 적용됩니다.

tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

In [ ]:
# 요약 문제는 sequence-to-sequence task이기 때문에 TFAutoModelForSeq2SeqLM 클래스를 통해 T5 모델을 로딩합니다.
# tokenizer와 마찬가지로 from_pretrained 메소드를 통해 T5 모델을 다운받을 수 있습니다.

from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq

model = TFAutoModelForSeq2SeqLM.from_pretrained(MODEL_CHECKPOINT, from_pt=True) # tf_model.h5 포멧의 경우는 from_pt=True 옵션 없어도 됨

In [ ]:
# Sequencce to Sequence 모델을 학습하기 위해서는 이에 맞는 data collator가 필요합니다.
# input뿐만 아니라 labels에 대해서도 padding처리를 해줘야 하기 때문입니다. 이를 위해 DataCollatorForSeq2Seq 를 사용할 수 있습니다.
# keras를 이용하고 있으므로 리턴 타입 tf.Tensor로 얻을 수 있도록 return_tensors="tf" 을 셋팅해줍니다.

from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

In [ ]:
# data_collator function을 적용해줍니다.
# 추가적으로 generation_dataset 이라는 작은 사이즈의 데이터셋을 학습시에 계산할 ROUGE score를 위해 추가로 생성해줍니다.

train_dataset = tokenized_datasets["train"].to_tf_dataset(
    batch_size=BATCH_SIZE,
    columns=["input_ids", "attention_mask", "labels"],
    shuffle=True,
    collate_fn=data_collator,
)
test_dataset = tokenized_datasets["test"].to_tf_dataset(
    batch_size=BATCH_SIZE,
    columns=["input_ids", "attention_mask", "labels"],
    shuffle=False,
    collate_fn=data_collator,
)
generation_dataset = (
    tokenized_datasets["test"]
    .shuffle()
    .select(list(range(200)))
    .to_tf_dataset(
        batch_size=BATCH_SIZE,
        columns=["input_ids", "attention_mask", "labels"],
        shuffle=False,
        collate_fn=data_collator,
    )
)

In [ ]:
# 이제 optimizer를 정의하고 모델을 컴파일 해줍니다. loss 계산은 내부적으로 처리됩니다.

optimizer = keras.optimizers.Adam(learning_rate=LEARNING_RATE)
model.compile(optimizer=optimizer)

In [ ]:
# 학습을 진행하는 동안 동시에 모델을 평가하기 위해 
# metric_fn을 정의해서 ground-truth와 prediction 간의 ROUGE score를 계산해보겠습니다.

import keras_nlp

rouge_l = keras_nlp.metrics.RougeL()


def metric_fn(eval_predictions):
    predictions, labels = eval_predictions
    decoded_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    for label in labels:
        label[label < 0] = tokenizer.pad_token_id  # masked label tokens 교체
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    result = rouge_l(decoded_labels, decoded_predictions)
    # 다른 metrics도 추가 가능하지만, 여기에서는 f1_score만 출력되도록 셋팅
    result = {"RougeL": result["f1_score"]}

    return result

In [ ]:
# 모델 학습을 시작하는 부분입니다.

from transformers.keras_callbacks import KerasMetricCallback

metric_callback = KerasMetricCallback(
    metric_fn, eval_dataset=generation_dataset, predict_with_generate=True
)

callbacks = [metric_callback]

# test set을 validation set으로 사용해보겠습니다
model.fit(
    train_dataset, validation_data=test_dataset, epochs=MAX_EPOCHS, callbacks=callbacks
)
     

In [ ]:
# Inference 부분입니다.
# 학습한 모델로 inference를 해보겠습니다. 
# Huggingface의 pipeline 메소드에서 제공하는 summarization을 사용하면 쉽게 구현할 수 있습니다.
# pipeline 메소드에 학습한 모델과 토크나이저를 인자로 넣어줍니다. 
# 모델이 TF로 학습되었으므로 framework="tf"도 함께 인자로 넣어줍니다.

from transformers import pipeline

summarizer = pipeline("summarization", model=model, tokenizer=tokenizer, framework="tf")

summary = summarizer(
              raw_datasets["test"][0]["document"],
              min_length=MIN_TARGET_LENGTH,
              max_length=MAX_TARGET_LENGTH,
          )

print(f'document:{raw_datasets["test"][0]["document"]}')
print(f'label summary:{raw_datasets["test"][0]["summary"]}')
print(f'pred summary: {summary[0]["summary_text"]}')

In [ ]:
[raw_datasets["test"][0]["summary"].split()]

In [ ]:
summary[0]["summary_text"].split()

In [ ]:
import nltk.translate.bleu_score as bleu
print('BLEU Score:',bleu.sentence_bleu([raw_datasets["test"][0]["summary"].split()],summary[0]["summary_text"].split()))

In [ ]:
"""
slist = []
for i in range(10):
    slist.append(input("input string:"))

for i in range(10):
    summarizer = pipeline("summarization", model=model, tokenizer=tokenizer, framework="tf")

    summary = summarizer(
                slist[i],
                min_length=MIN_TARGET_LENGTH,
                max_length=MAX_TARGET_LENGTH,
            )
    print(f'pred summary: {summary[0]["summary_text"]}')

    slist = []
"""


## 여기서 문장 길게 쓰신 다음 결과 보시면 됩니다
sentence = input("input string:"))
summarizer = pipeline("summarization", model=model, tokenizer=tokenizer, framework="tf")
summary = summarizer(
            sentence,
            min_length=MIN_TARGET_LENGTH,
            max_length=MAX_TARGET_LENGTH,
        )
print(f'pred summary: {summary[0]["summary_text"]}')